In [2]:

import numpy as np 
import matplotlib.pyplot as plt 
import math
from scipy.spatial.transform import Rotation as R
from ipywidgets import interact, fixed
from itertools import product, combinations, permutations
from IPython.display import clear_output
import warnings

import sympy as sp

In [5]:
def uvw2hkl(uvw, lattice_vector, multiply=10):
    
    uvw = uvw.flatten()
    uvw = uvw@lattice_vector
    u, v, w = uvw/np.linalg.norm(uvw) + 1e-8
    hkl_candidate = []
    
    try:
        ans_h = -29428309.5638271*a*u*math.sqrt(-w/(u*(500000000000000.0*u - 866025403784439.0*v)))/w
        ans_k = 2.94283095638271e-8*a*math.sqrt(-w/(u*(500000000000000.0*u - 866025403784439.0*v)))*(500000000000000.0*u - 866025403784439.0*v)/w
        ans_l = -29428309.5638271*c*math.sqrt(-w/(u*(500000000000000.0*u - 866025403784439.0*v)))
        hkl_candidate.append([ans_h, ans_k, ans_l])
    except:
        pass

    try:
        ans_h = -29428309.5638271*a*u*math.sqrt(w/(u*(500000000000000.0*u - 866025403784439.0*v)))/w
        ans_k = 2.94283095638271e-8*a*math.sqrt(w/(u*(500000000000000.0*u - 866025403784439.0*v)))*(500000000000000.0*u - 866025403784439.0*v)/w
        ans_l = -29428309.5638271*c*math.sqrt(w/(u*(500000000000000.0*u - 866025403784439.0*v)))
        hkl_candidate.append([ans_h, ans_k, ans_l])
    except:
        pass
    
    try:
        ans_h = 29428309.5638271*a*u*math.sqrt(-w/(u*(500000000000000.0*u - 866025403784439.0*v)))/w
        ans_k = 2.94283095638271e-8*a*math.sqrt(-w/(u*(500000000000000.0*u - 866025403784439.0*v)))*(-500000000000000.0*u + 866025403784439.0*v)/w
        ans_l = 29428309.5638271*c*math.sqrt(-w/(u*(500000000000000.0*u - 866025403784439.0*v)))
        hkl_candidate.append([ans_h, ans_k, ans_l])
    except:
        pass

    try:
        ans_h = 29428309.5638271*a*u*math.sqrt(w/(u*(500000000000000.0*u - 866025403784439.0*v)))/w
        ans_k = 2.94283095638271e-8*a*math.sqrt(w/(u*(500000000000000.0*u - 866025403784439.0*v)))*(-500000000000000.0*u + 866025403784439.0*v)/w
        ans_l = 29428309.5638271*c*math.sqrt(w/(u*(500000000000000.0*u - 866025403784439.0*v)))
        hkl_candidate.append([ans_h, ans_k, ans_l])
    except:
        pass
    
    inner_prod_list = [uvw@hkl for hkl in hkl_candidate]
    max_sim_hkl = hkl_candidate[np.argmax(inner_prod_list)]
    
    hkl_denote = str(np.round(max_sim_hkl/np.linalg.norm(max_sim_hkl)*multiply, 1))
    max_sim_hkl = max_sim_hkl/np.linalg.norm(max_sim_hkl)
    return hkl_denote, max_sim_hkl, hkl_candidate

def norm_vector(n_, v_):
    p = n_ - np.dot(n_, v_.T) * v_
    return p

def rotate(x, y, z, vector):
    rx = np.array([[1, 0, 0], [0, np.cos(x), -np.sin(x)], [0, np.sin(x), np.cos(x)]])
    ry = np.array([[np.cos(y), 0, np.sin(y)], [0, 1, 0], [-np.sin(y), 0, np.cos(y)]])
    rz = np.array([[np.cos(z), -np.sin(z), 0], [np.sin(z), np.cos(z), 0], [0, 0, 1]])
    vector_ = np.dot(rz, np.dot(ry, np.dot(rx, vector)))
    
    return vector_ 

def angle_vector(vector1, vector2):
    dot_product = np.dot(vector1, vector2.T).T
    magnitude1 = np.linalg.norm(vector1, axis=1)
    magnitude2 = np.linalg.norm(vector2)
    angle = np.arccos(dot_product / (magnitude1 * magnitude2))
    angle_degrees = np.rad2deg(angle)
    return angle, angle_degrees

def angle_project_calc(v_plane, v1, v2):
    p1 = norm_vector(v1, v_plane)
    p2 = norm_vector(v2, v_plane)
    angle_project = angle_vector(p1, p2)
    return angle_project

def proj_coord(vector, r):
     #* Norm vector projecting on the viewpoint axis
    norm_111 = norm_vector(vector, np.array([[0,0,1]])) #* Vector here is the 'true' vector after rotation from phi, theta, psi
    #* Angle between the norm vector and the viewpoint axis
    rho, _ = angle_vector(vector, np.array([[0,0,1]]))
    d = r*np.tan(rho/2)
    unit_len = np.sqrt(d**2/np.sum(norm_111**2, axis=1)).T
    norm_111_ = norm_111*unit_len
    x_len, y_len, z_len = norm_111_[:,0], norm_111_[:,1], norm_111_[:,2]
    
    if np.any(np.isnan([x_len[0], y_len[0]])):
        x_len = [0.]
        y_len = [0.]
        
    return x_len, y_len

def rotation_axis(vector1, vector2):
    axis = np.cross(vector1, vector2)
    return axis

def euler_angle_calc(rotate_axis, rotate_degree):
    quaternion = R.from_rotvec(rotate_axis * rotate_degree / np.linalg.norm(rotate_axis))
    # quaternion_conj = R.from_rotvec(rotate_axis * -rotate_degree / np.linalg.norm(rotate_axis)) #* Minus rotation for vector buffer
    euler_angles = quaternion.as_euler('zyx',)
    # euler_angles_conj = quaternion_conj.as_euler('zyx',)
    
    return euler_angles

def frac2cart(vec, lattice_vector):
    '''
    Converts indices to cartesian coordinates
    '''
    return np.dot(vec, lattice_vector)

def cart2frac(vec, lattice_vector):
    '''
    Converts indices to fractional coordinates
    '''
    return np.dot(vec, np.linalg.inv(lattice_vector))
    
def hkl2uvw(hkl, lattice_vector):
    hkl = hkl.flatten()
    h, k, l = hkl
    
    def inf_convert(x):
        if x == 0:
            return 1e-8
        else:
            return x
        
    v1_frac, v2_frac = np.array([1/inf_convert(h),-1/inf_convert(k),0]), np.array([1/inf_convert(h),0,-1/inf_convert(l)])
    v1_cart, v2_cart = frac2cart(v1_frac, lattice_vector), frac2cart(v2_frac, lattice_vector)
    v_cross = np.cross(v1_cart, v2_cart)
    v_frac_1 = cart2frac(v_cross, lattice_vector)
    v_frac_2 = -v_frac_1
    
    v_frac_1, v_frac_2 = v_frac_1/np.linalg.norm(v_frac_1), v_frac_2/np.linalg.norm(v_frac_2)
    #* return the closest vector to the target vector
    if np.dot(v_frac_1, hkl) > np.dot(v_frac_2, hkl):
        return v_frac_1
    else:
        return v_frac_2

def inv_point(x, y, rot, lattice_vector, multiply=1, decimal=3):
    '''return the lattice point by clicking on the projection plane'''
    d = np.sqrt(x**2 + y**2)
    z = 2/(1+d**2)-1 #* cos(rho)
    try:
        x_true = np.sqrt((1-z**2)/(1+(y**2/x**2)))*np.sign(x)
        y_true = (y/x)*x_true
    except:
        x_true = 0
        y_true = 0
    
    # print([x, y, x_true, y_true, z])
    convert_point = rot.apply(
        np.array([x_true, y_true, z]))@np.linalg.inv(lattice_vector)
      
    convert_point = convert_point/np.linalg.norm(convert_point)
    return str(np.round(convert_point*multiply, decimal)), convert_point

def emission_point(uvw, length):
    '''  
    Return emission coordinate by [uvw] and 1/lambda=length to generate
    Ewald sphere
    '''
    u, v, w = uvw[0]
    x = np.sqrt(length**2/(1+v**2/u**2+w**2/u**2))
    y = v/u*x
    z = w/u*x
    
    return np.array([x,y,z])
    
r = 1
x_010, y_100, symbol = [], [], []

#! Input the crystal information in the following format 
#* Modify it when necessary
#* Transfer the coordinates
# lattice_vector_real = np.array([3.16,3.16,3.52]) #* EBSD profile's information
lattice_vector_real = np.array([3.124,3.124,3.387]) #* custom lattice profile
a, b, c = lattice_vector_real
alpha_rad = np.deg2rad(90)
beta_rad = np.deg2rad(90)
gamma_rad = np.deg2rad(120)

c31 = c * np.cos(beta_rad)
c32 = c * (np.cos(alpha_rad) - np.cos(beta_rad) * np.cos(gamma_rad)) / np.sin(gamma_rad)
c33 = c * np.sqrt(1 - np.cos(alpha_rad)**2 
                  - np.cos(beta_rad)**2 - np.cos(gamma_rad)**2
                  + 2 * np.cos(alpha_rad) * np.cos(beta_rad) * np.cos(gamma_rad)) \
                      / np.sin(gamma_rad)

lattice_vector = np.array([
    [a, 0, 0],
    [b * np.cos(gamma_rad), b * np.sin(gamma_rad), 0],
    [c31, c32, c33]
])

longitude = np.load(r'utils/longitude.npy')
latitude = np.load(r'utils/latitude.npy')

#* Real space preparation
# vector_buffer_raw = np.load(r'./utils/vector_buffer_raw.npy')
vector_buffer_raw = np.array(list(product(range(-3,4), repeat=3)))

#* generate the hkl vector buffer
vector_buffer_raw_hkl = np.array([hkl2uvw(vec_raw, lattice_vector) for vec_raw in vector_buffer_raw])

vector_buffer = vector_buffer_raw.copy()
vector_buffer_ = vector_buffer@lattice_vector
vector_buffer_ = vector_buffer_/np.linalg.norm(vector_buffer_, axis=1)[:, None]

vector_buffer_hkl = vector_buffer_raw_hkl.copy()
vector_buffer_hkl_ = vector_buffer_hkl@lattice_vector
vector_buffer_hkl_ = vector_buffer_hkl_/np.linalg.norm(vector_buffer_hkl_, axis=1)[:, None]

viewpoint_axis = np.array([[0,0,1]]) #* Origin viewpoint axis
viewpoint_axis_ = viewpoint_axis@lattice_vector
viewpoint_axis_ = viewpoint_axis_/np.linalg.norm(viewpoint_axis_)

#TODO specify the target vector
special_vector = np.array([
                        # [6, -42, -7], #* uvw for grain 
                        # [3,0,3],[-3, 0, -3],
                        # [3,3,-3],[-3,-3,3],
                        [7.5, 5.6, 3.6],
                        # [-3, 0, -3],
                        ]) #* Special vector for displaying on projection plane
special_vector_ = special_vector@lattice_vector
special_vector_ = special_vector_/np.linalg.norm(special_vector_)

uvtw_mat = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [-1, -1, 0],
    [0, 0, 1]
])

# target_vector = np.array([[-18,  15,  13]])@np.linalg.inv(lattice_vector@lattice_vector.T)+1e-8 #* Calibrated by lattice vector
# target_vector = np.array([0, 0, 1])+1e-8 #* Target for rotating
# target_vector = np.array([[-9.5, -2.2, 2.2]])+1e-8 #* Target for rotating
#TODO if need to convert hkl to uvw
target_vector = np.array([[0,0,1]])+1e-8 #* Target for rotating
# target_vector = hkl2uvw(target_vector, lattice_vector).reshape(1,-1)
print(f'raw target vector: {target_vector}')

target_vector_ = target_vector@lattice_vector
target_vector_ = target_vector_/np.linalg.norm(target_vector_)

rotate_axis = rotation_axis(viewpoint_axis_, target_vector_)
rotate_degree = angle_vector(viewpoint_axis_, target_vector_)[0]
euler_angle = euler_angle_calc(rotate_axis, rotate_degree)
rot = R.from_euler('zyx', euler_angle)
rot_conj = rot.inv()

vector_buffer_ = rot_conj.apply(vector_buffer_)
vector_buffer_hkl_ = rot_conj.apply(vector_buffer_hkl_)
special_vector_ = rot_conj.apply(special_vector_)

#* Reciprocal space preparation
#* A first rotate will be applied as well
vector_buffer_uvw = np.load(r'utils/hkl_list/alb2_hkl_list.npy')

#! Rotate the vector buffer to reach the target projection plane.
def main(vector_buffer_raw,
         vector_buffer_raw_hkl,
         special_vector_raw, 
         target_vector_raw,
         rot_raw,
         phi=-166, theta=-1e-8, psi=-1e-8,
         x_view=1e-8, y_view=1e-8,
         x_=1e-8,y_=1e-8,
         x_2=1e-8,y_2=1e-8,
         phi_rot=90, phi_rot2=180):
    
    y_100, x_010, symbol = [], [], []
    y_100_special, x_010_special, symbol_special = [], [], []


    #TODO Modify 231007: addition of (x_view, y_view) for the view point
    #! the reference coordinates are still in fcc coordinate? -> so it's not [1 0 0; 0 1 0; 0 0 1] but newly calculated one required.
    #* deduce rot_xy from (x_view, y_view) in the unit of degree.
    euler_xy = np.array([0, x_view, y_view])
    rot_xy = R.from_euler('zyx', euler_xy, degrees=True)
    proj_001 = rot_xy.apply(np.array([[0,0,1]]))
    #* Calculate the projection of proj_001
    proj_001_x, proj_001_y = proj_coord(proj_001, r)
    
    centre_raw = inv_point(0, 0, rot_raw, lattice_vector)[1]
    centre_raw = centre_raw/np.linalg.norm(centre_raw)
    centre_xy = inv_point(proj_001_x[0]+1e-8, proj_001_y[0]+1e-8, rot_raw, lattice_vector)[1]
    centre_xy = centre_xy/np.linalg.norm(centre_xy)
    #* rotation matrix from `target_vector` to `centre_xy`
    rot_xy = R.from_rotvec(np.cross(centre_raw, centre_xy)).inv()
      
    #* For true-rotation
    euler_angle = np.array([phi,theta,psi])
    rot = R.from_euler('zyx', euler_angle, degrees=True)
    
    rot = rot_xy*rot #TODO addition of rot_xy
    rot_inv = rot.inv()
    # rot_conj_inv = rot_conj.inv()
    vector_buffer_ = rot_inv.apply(vector_buffer_raw) #* After rotation from phi, theta, psi
    vector_buffer_hkl_ = rot_inv.apply(vector_buffer_raw_hkl) #* After rotation from phi, theta, psi
    special_vector_ = rot_inv.apply(special_vector_raw)
    target_vector_ = rot.apply(target_vector_raw)
    
    rot_combineall = rot_raw*rot
    
    click_pole, click_pole_vec = inv_point(x_, y_, rot_combineall, lattice_vector, multiply=10, decimal=1)
    click_pole_2nd, click_pole_2nd_vec = inv_point(x_2, y_2, rot_combineall, lattice_vector, multiply=10, decimal=1)
    click_pole_hkl = uvw2hkl(click_pole_vec, lattice_vector)[0]
    click_pole_2nd_hkl = uvw2hkl(click_pole_2nd_vec, lattice_vector)[0]
    
    #* point on unit circle defined by input value of phi
    x_rot, y_rot = 1*np.sin(np.deg2rad(phi_rot)), 1*np.cos(np.deg2rad(phi_rot))
    click_pole_rot, click_pole_rot_vec = inv_point(x_rot, y_rot, rot_combineall, lattice_vector, multiply=10, decimal=1)
    click_pole_rot_hkl = uvw2hkl(click_pole_rot_vec, lattice_vector)[0]
    
    x_rot_2, y_rot_2 = 1*np.sin(np.deg2rad(phi_rot2)), 1*np.cos(np.deg2rad(phi_rot2))
    click_pole_rot_2, click_pole_rot_2_vec = inv_point(x_rot_2, y_rot_2, rot_combineall, lattice_vector, multiply=10, decimal=1)
    click_pole_rot_2_hkl = uvw2hkl(click_pole_rot_2_vec, lattice_vector)[0]
    
    central_axis = inv_point(0, 0, rot_combineall, lattice_vector)[1]
    central_axis_hkl = uvw2hkl(central_axis, lattice_vector)[1]

    x_010, y_100 = proj_coord(vector_buffer_, r) #* Projection on the plane, for uvw
    x_010_hkl, y_100_hkl = proj_coord(vector_buffer_hkl_, r) #* Projection on the plane, for hkl
    symbol = [str(vector_raw) for vector_raw in vector_buffer]
        
    pole_list = np.array([x_010, y_100]).T
    pole_list_hkl = np.array([x_010_hkl, y_100_hkl]).T
    valid_pole_ind = np.where(np.linalg.norm(pole_list, axis=1) <= 1.4)[0]
    valid_pole = pole_list[valid_pole_ind]
    valid_pole_hkl = pole_list_hkl[valid_pole_ind]
        
    x_010_special, y_100_special = proj_coord(special_vector_, r)
    symbol_special = [str(vector_raw) for vector_raw in special_vector]

    pole_list_special = np.array([x_010_special, y_100_special]).T
    valid_pole_ind_special = np.where(np.linalg.norm(pole_list_special, axis=1) <= 1.4)[0]
    valid_pole_special = pole_list_special[valid_pole_ind_special]
    
    # print(vector_buffer_reci_, vector_buffer_reci_diff_)
    clear_output(wait=True)
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(40, 20))
    
    #* Real space
    ax1.scatter(x_, y_, s=200, c='green', marker='o')
    ax1.text(x_, y_, str(click_pole), fontsize=20)

    ax2.scatter(x_, y_, s=200, c='green', marker='o')
    ax2.text(x_, y_, str(click_pole_hkl), fontsize=20)
     
    ax1.scatter(x_2, y_2, s=200, c='green', marker='o')
    ax1.text(x_2, y_2, str(click_pole_2nd), fontsize=20)

    ax2.scatter(x_2, y_2, s=200, c='green', marker='o')
    ax2.text(x_2, y_2, str(click_pole_2nd_hkl), fontsize=20)
    
    ax1.scatter(x_rot, y_rot, s=200, c='k', marker='o')
    ax1.text(x_rot, y_rot, str(click_pole_rot), fontsize=20)

    ax2.scatter(x_rot, y_rot, s=200, c='k', marker='o')
    ax2.text(x_rot, y_rot, str(click_pole_rot_hkl), fontsize=20)
    
    ax1.scatter(x_rot_2, y_rot_2, s=200, c='k', marker='o')
    ax1.text(x_rot_2, y_rot_2, str(click_pole_rot_2), fontsize=20)

    ax2.scatter(x_rot_2, y_rot_2, s=200, c='k', marker='o')
    ax2.text(x_rot_2, y_rot_2, str(click_pole_rot_2_hkl), fontsize=20)
    
    ax1.scatter(valid_pole[:,0], valid_pole[:,1], s=100, c='orangered', marker='o')
    ax2.scatter(valid_pole_hkl[:,0], valid_pole_hkl[:,1], s=100, c='cornflowerblue', marker='o')

    ax1.scatter(valid_pole_special[:,0], valid_pole_special[:,1], s=200, c='b', marker='o')

    ax1.set_aspect('equal')
    ax1.set_xlim(-1.4, 1.4)
    ax1.set_ylim(-1.4, 1.4)
    
    ax2.set_aspect('equal')
    ax2.set_xlim(-1.4, 1.4)
    ax2.set_ylim(-1.4, 1.4)
    
    for i in valid_pole_ind:
        ax1.text(x_010[i], y_100[i], symbol[i], fontsize=10)
        ax2.text(x_010_hkl[i], y_100_hkl[i], symbol[i], fontsize=10)
    for i in valid_pole_ind_special:
        ax1.text(x_010_special[i], y_100_special[i], symbol_special[i], fontsize=20)
        # ax2.text(x_010_special[i], y_100_special[i], symbol_special[i], fontsize=20)

    #* Display the background 
    ax1.scatter(longitude[:,0], longitude[:,1], 
                s=0.01, c='#838B8B', marker='o', zorder=0, alpha=0.2)
    ax1.scatter(latitude[:,0], latitude[:,1], 
                s=0.01, c='#838B8B', marker='o', zorder=0, alpha=0.2)

    ax2.scatter(longitude[:,0], longitude[:,1], 
                s=0.01, c='#838B8B', marker='o', zorder=0, alpha=0.2)
    ax2.scatter(latitude[:,0], latitude[:,1], 
                s=0.01, c='#838B8B', marker='o', zorder=0, alpha=0.2)
    
    #* display text
    ax1.text(0.52, 1.25, 'UVW projection', fontsize=40)
    ax2.text(0.52, 1.25, 'HKL projection', fontsize=40)
    
    plt.title(f'{np.round(central_axis, 3)} central axis in uvw\n {np.round(central_axis_hkl, 3)} central axis in hkl', fontsize=20)
    plt.show()


warnings.simplefilter('ignore', category=RuntimeWarning)
interact(main,
        vector_buffer_raw = fixed(vector_buffer_),
        vector_buffer_raw_hkl = fixed(vector_buffer_hkl_),
        special_vector_raw = fixed(special_vector_),
        target_vector_raw = fixed(target_vector_),
        rot_raw = fixed(rot),
        phi = (-180, 180, 2),
        theta = (-180, 180, 2),
        psi = (-180, 180, 2), 
        x_view = (-90, 90, 2),
        y_view = (-90, 90, 2),
        x_ = (-1.4, 1.4, 0.01),
        y_ = (-1.4, 1.4, 0.01),
        x_2 = (-1.4, 1.4, 0.01),
        y_2 = (-1.4, 1.4, 0.01),
        phi_rot= (0, 360, 1),
        phi_rot2= (0, 360, 1),
)


raw target vector: [[1.00000000e-08 1.00000000e-08 1.00000001e+00]]


interactive(children=(IntSlider(value=-166, description='phi', max=180, min=-180, step=2), IntSlider(value=0, …

<function __main__.main(vector_buffer_raw, vector_buffer_raw_hkl, special_vector_raw, target_vector_raw, rot_raw, phi=-166, theta=-1e-08, psi=-1e-08, x_view=1e-08, y_view=1e-08, x_=1e-08, y_=1e-08, x_2=1e-08, y_2=1e-08, phi_rot=90, phi_rot2=180)>

Cross product of two detected vectors.

In [23]:
v1, v2 = np.array([-7.4, 6.7, 0]), np.array([-1,-2,0])
v1_real, v2_real = v1@lattice_vector, v2@lattice_vector 
v_cross = np.cross(v1_real, v2_real)@np.linalg.inv(lattice_vector)

print(v_cross, uvw2uvtw(v_cross))

[-7.12345965e-15 -7.12345965e-15  5.36507683e+01] [-2.37448655e-15 -2.37448655e-15  4.74897310e-15  5.36507683e+01]


In [33]:
v_true = v_cross@lattice_vector

perfect_hkl = np.array([1, 2, 0])
v_ideal = perfect_hkl@lattice_vector

theta = np.arccos(v_true@v_ideal/(np.linalg.norm(v_true)*np.linalg.norm(v_ideal)))
np.rad2deg(theta)

147.74298969517199